<a href="https://colab.research.google.com/github/akshaymargaj/sqlprojects/blob/main/DB_CW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Main Script

In [ ]:
import pandas as pd
import warnings
import os
from google.colab import files
import re

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Install openpyxl if not already installed
!pip install openpyxl

# Define the path to the input Excel file
excel_path = "/content/DATA.xlsx"
if not os.path.exists(excel_path):
    raise FileNotFoundError(f"Input Excel file not found at: {excel_path}. Please upload 'Punchuality_Stats_unpivoted.xlsx' to the Files tab.")

# Load Excel data (assuming data is in the first sheet)
try:
    df = pd.read_excel(excel_path, engine='openpyxl')
except Exception as e:
    raise Exception(f"Failed to load Excel file: {str(e)}. Check file format, sheet name, or data integrity.")

# Print column names for debugging
print("Loaded columns:", list(df.columns))

# Verify required columns exist
required_columns = ['REPORTING_AIRPORT', 'ORIGIN_DESTINATION_COUNTRY', 'ORIGIN_DESTINATION', 'AIRLINE_NAME', 'value']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Required columns missing in Excel file: {missing_columns}")

# Ensure 'value' is numeric
df['value'] = pd.to_numeric(df['value'], errors='coerce').fillna(0)

# Collect unique values for diagnostics
unique_airports = set(df['REPORTING_AIRPORT'].dropna().unique()) | set(df['ORIGIN_DESTINATION'].dropna().unique())
unique_countries = set(df['ORIGIN_DESTINATION_COUNTRY'].dropna().unique())
unique_airlines = set(df['AIRLINE_NAME'].dropna().unique())

# Define airport name to IATA code mapping (expanded)
airport_mapping = {
    'aberdeen': 'ABZ',
    'belfast city (george best)': 'BHD',
    'belfast city': 'BHD',
    'belfast international': 'BFS',
    'newcastle': 'NCL',
    'southampton': 'SOU',
    'southend': 'SEN',
    'teesside international airport': 'MME',
    'teesside': 'MME',
    'alderney': 'ACI',
    'alicante': 'ALC',
    'amsterdam': 'AMS',
    'antalya': 'AYT',
    'arrecife': 'ACE',
    'athens': 'ATH',
    'barcelona': 'BCN',
    'beauvais': 'BVA',
    'bergen': 'BGO',
    'bergamo': 'BGY',
    'birmingham': 'BHX',
    'bodrum (milas)': 'BJV',
    'bristol': 'BRS',
    'budapest': 'BUD',
    'burgas': 'BOJ',
    'caen': 'CFR',
    'cardiff wales': 'CWL',
    'catania (fontanarossa)': 'CTA',
    'chania': 'CHQ',
    'corfu': 'CFU',
    'BOSNIA-HERZEGOVINA': 'BIH',
    'copenhagen': 'CPH',
    'dalaman': 'DLM',
    'dublin': 'DUB',
    'dundee': 'DND',
    'east midlands international': 'EMA',
    'edinburgh': 'EDI',
    'esbjerg': 'EBJ',
    'exeter': 'EXT',
    'faro': 'FAO',
    'gatwick': 'LGW',
    'gdansk': 'GDN',
    'glasgow': 'GLA',
    'guernsey': 'GCI',
    'humberside': 'HUY',
    'ibiza': 'IBZ',
    'inverness': 'INV',
    'isle of man': 'IOM',
    'jersey': 'JER',
    'kefallinia': 'EFL',
    'kirkwall': 'KOI',
    'kos': 'KGS',
    'krakow': 'KRK',
    'larnaca': 'LCA',
    'leeds bradford': 'LBA',
    'liverpool (john lennon)': 'LPL',
    'london city': 'LCY',
    'luton': 'LTN',
    'mahon': 'MAH',
    'malaga': 'AGP',
    'manchester': 'MAN',
    'newquay': 'NQY',
    'norwich': 'NWI',
    'OSLO (GARDERMOEN)': 'OSL',
    'palma de mallorca': 'PMI',
    'paphos': 'PFO',
    'rhodes': 'RHO',
    'salonika': 'SKG',
    'skiathos': 'JSI',
    'stansted': 'STN',
    'sumburgh': 'LSI',
    'TENERIFE (SURREINA SOFIA)': 'TFS',
    'toronto': 'YYZ',
    'wick john o groats': 'WIC',
    'zakinthos': 'ZTH',
    'eindhoven': 'EIN',
    'heathrow': 'LHR',
    # Additional airports for broader coverage
    'bournemouth': 'BOH',
    'cardiff': 'CWL',
    'doncaster sheffield': 'DSA',
    'durham tees valley': 'MME',
    'leeds': 'LBA',
    'liverpool': 'LPL',
    'london gatwick': 'LGW',
    'london heathrow': 'LHR',
    'london luton': 'LTN',
    'london stansted': 'STN',
    'prestwick': 'PIK',
    'crete chania': 'CHQ',
    'crete heraklion': 'HER',
    'fuerteventura': 'FUE',
    'gran canaria': 'LPA',
    'lanzarote': 'ACE',
    'lisbon': 'LIS',
    'madeira': 'FNC',
    'murcia': 'RMU',
    'nice': 'NCE',
    'paris charles de gaulle': 'CDG',
    'paris orly': 'ORY',
    'prague': 'PRG',
    'reykjavik keflavik': 'KEF',
    'rome fiumicino': 'FCO',
    'santorini': 'JTR',
    'sofia': 'SOF',
    'tenerife south': 'TFS',
    'venice marco polo': 'VCE',
    'warsaw chopin': 'WAW',
    'lublin port lotniczy': 'LUZ',
    'erbil international': 'EBL',
    'varadero': 'VRA',
    'sao paulo guarulhos': 'GRU',
    'tbilisi': 'TBS',
    'ho chi minh city': 'SGN',
    'bale mulhouse': 'MLH',
    'bodrum milas': 'BJV',
    'sulaymaniyah int': 'ISU',
    'e t joshua': 'SVD',  # Renamed to Argyle International in some contexts
    'akureyri': 'AEY',
    'dole': 'DLE',
    'puerto vallarta': 'PVR',
    'detroit': 'DTW',
    'port sudan': 'PZU',
    'bandar seri begawan': 'BWN',
    'guangzhou baiyun international': 'CAN',
    'forli': 'FRL',
    'essaouira': 'ESU',
    'perth australia': 'PER',
    'bacau': 'BCM',
    'odense': 'ODE',
    'bologna': 'BLQ',
    'oslo gardermoen': 'OSL',
    'sylhet': 'ZYL',
    'cairo west afb': 'CWE',  # Military base, no IATA, use abbreviation
    'gander': 'YQX',
    'biarritz': 'BIQ',
    'antwerp': 'ANR',
    'split': 'SPU',
    'orlando': 'MCO',
    'farnborough': 'FAB',
    'ireland westknock': 'NOC',
    'cork': 'ORK',
    'bahrain': 'BAH',
    'manila': 'MNL',
    'phuket': 'HKT',
    'vitoria': 'VIT',
    'ajaccio': 'AJA',
    'aqaba': 'AQJ',
    'cape town': 'CPT:',
    'ingolstadt-manching': 'IGS',
    'naples': 'NAP',
    'kaunas': 'KUN',
    'philadelphia international': 'PHL',
    'tampere': 'TMP',
    'grand cayman': 'GCM',
    'rovaniemi': 'RVN',
    'paris orly': 'ORY',
    'amritsar': 'ATQ',
    'reus': 'REU',
    'strasbourg': 'SXB',
    'durban': 'DUR',
    'dubai world central': 'DWC',
    'casablanca mohamed v': 'CMN',
    'helsinki': 'HEL',
    'reggio': 'REG',
    'ostend': 'OST',
    'kigali': 'KGL',
    'abuja': 'ABV',
    'praia': 'RAI',
    'warsaw chopin': 'WAW',
    'ramon': 'ETM',
    'ivalo': 'IVL',
    'doha international': 'DOH',  # Legacy, replaced by DOH (Hamad)
    'rzeszow': 'RZE',
    'vaxjo': 'VXO',
    'rome ciampino': 'CIA',
    'sion': 'SIR',
    'mytilini': 'MJT',
    'tiree': 'TRE',
    'bermuda': 'BDA',
    'zurich': 'ZRH',
    'sonderborg': 'SGD',
    'cannes': 'CEQ',
    'craiova': 'CRA',
    'kassel': 'KSF',
    'lodz lublinek': 'LCJ',
    'pau': 'PUF',
    'tivat': 'TIV',
    'perpignan': 'PGF',
    'turin': 'TRN',
    'catania fontanarossa': 'CTA',
    'TARBES-LOURDES INTERNATIONAL': 'LDE',
    'alta': 'ALF',
    'clermont ferrand': 'CFE',
    'tours': 'TUF',
    'bridgetown': 'BGI',
    'columbus': 'CMH',
    'tobago': 'TAB',
    'brest': 'BES',
    'la rochelle': 'LRH',
    'adana': 'ADA',
    'goteborg landvetter': 'GOT',
    'valence': 'VLC',  # Likely Valencia, Spain
    'baltimore': 'BWI',
    'yerevan': 'EVN',
    'johannesburg': 'JNB',
    'malta': 'MLA',
    'lulea': 'LLA',
    'enontekio': 'ENF',
    'harare': 'HRE',
    'dhakha': 'DAC',  # Likely typo for Dhaka
    'nuremberg': 'NUE',
    'satu mare': 'SUJ',
    'longyearbyen svalbard': 'LYR',
    'buenos aires': 'EZE',  # Likely Ministro Pistarini
    'vilnius': 'VNO',
    'samsun carsamba': 'SZF',
    'hyderabad rajiv ghandi': 'HYD',
    'almaty': 'ALA',
    'salamanca': 'SLM',
    'chisinau international': 'KIV',
    'sarajevo': 'SJJ',
    'beirut': 'BEY',
    'munster-osnabruck': 'FMO',
    'tenerife surreina sofia': 'TFS',
    'istanbul sabiha gokcen': 'SAW',
    'kuusamo': 'KAO',
    'beijing': 'PEK',  # Likely Beijing Capital
    'cincinnati': 'CVG',
    'belgrade': 'BEG',
    'ilha do sal cverde': 'SID',
    'brindisi': 'BDS',
    'bardufoss': 'BDU',
    'portland oregon': 'PDX',
    'ROSTOCK-LAAGE': 'RLG',
    'le touquet': 'LTQ',
    'valencia': 'VLC',
    'lahore': 'LHE',
    'arvidsjaur': 'AJR',
    'minneapolis-st paul': 'MSP',
    'oakland': 'OAK',
    'cascais': 'CAT',  # Small airfield, no standard IATA
    'haikou': 'HAK',
    'maastricht': 'MST',
    'tampa': 'TPA',
    'changsha huanghua international airport': 'CSX',
    'bogota': 'BOG',
    'goa international': 'GOI',
    'milan malpensa': 'MXP',
    'dallasfort worth': 'DFW',
    'kalamata': 'KLX',
    'kuala lumpur sepang': 'KUL',
    'sibiu': 'SBZ',
    'pamplona': 'PNA',
    'nairobi': 'NBO',
    'munich': 'MUC',
    'wroclaw': 'WRO',
    'darwin': 'DRW',
    'braunschweig': 'BWE',
    'skopje': 'SKP',
    'chengdu': 'CTU',  # Likely Chengdu Shuangliu
    'mulhouse habsheim': 'MLH',  # Same as Bale Mulhouse
    'innsbruck': 'INN',
    'johan adolf pengel': 'PBM',  # Suriname
    'baden baden': 'FKB',
    'trondheim vaernes': 'TRD',
    'marseille': 'MRS',
    'islamabad international airport': 'ISB',
    'melbourne': 'MEL',
    'punta cana': 'PUJ',
    'hamburg firkenwerder': 'XFW',  # Airbus facility
    'oxford kidlington': 'OXF',
    'san sebastian': 'EAS',
    'grenada': 'GND',
    'ouarzazate': 'OZZ',
    'muscat': 'MCT',
    'doha hamad': 'DOH',
    'green bay wisconsin': 'GRB',
    'tel aviv': 'TLV',
    'tabuk': 'TUU',
    'errachidia': 'ERH',
    'sandefjordtorp': 'TRF',
    'riyadh': 'RUH',
    'marhamkings lynn': 'KNF',  # RAF Marham
    'toulon  hyeres': 'TLN',
    'vigo': 'VGO',
    'brive-la-gaillarde': 'BVE',
    'jerez': 'XRY',
    'paris charles de gaulle': 'CDG',
    'memmingen allgau': 'FMM',
    'sarmellekbalaton': 'SOB',
    'palanga': 'PLQ',
    'warsaw modlin masovia': 'WMI',
    'amman': 'AMM',
    'pajala yllas': 'PJA',
    'moline quad city': 'MLI',
    'siauliai': 'SQQ',
    'van nuys': 'VNY',
    'nantes': 'NTE',
    'diyarbakir': 'DIY',
    'tromsoe': 'TOS',
    'rennes': 'RNS',
    'nanjing lukou international': 'NKG',
    'teterboro': 'TEB',
    'gibraltar': 'GIB',
    'seoul incheon': 'ICN',
    'aruba': 'AUA',
    'mostar': 'OMO',
    'bydgoszcz': 'BZG',
    'boa vista rabil': 'BVC',
    'bordeaux': 'BOD',
    'chisinau kishinev': 'KIV',
    'nevsehir kapadokya': 'NAV',
    'stavanger': 'SVG',
    'accra': 'ACC',
    'florence': 'FLR',
    'samos': 'SMI',
    'kerry county': 'KIR',
    'ashkhabad': 'ASB',
    'minot international': 'MOT',
    'scandinavian mountains airport': 'SCR',
    'osijek': 'OSI',
    'seville': 'SVQ',
    'beziers': 'BZR',
    'porto santo': 'PXO',
    'la romana': 'LRM',
    'billund': 'BLL',
    'salt lake city': 'SLC',
    'vienna': 'VIE',
    'riga': 'RIX',
    'girona': 'GRO',
    'verona villafranca': 'VRN',
    'zagreb': 'ZAG',
    'lametia-terme': 'SUF',
    'akrotiri': 'AKT',  # RAF Akrotiri, military
    'kiev zhulyany': 'IEV',
    'venice': 'VCE',
    'varna': 'VAR',
    'washington dulles': 'IAD',
    'bangalore bengaluru': 'BLR',
    'szymany mazury': 'SZY',
    'pardubice': 'PED',
    'thira santorini': 'JTR',
    'frankfurt main': 'FRA',
    'almeria': 'LEI',
    'nassau': 'NAS',
    'marsa alam': 'RMF',
    'cambridge': 'CBG',
    'benbecula': 'BEB',
    'miami international': 'MIA',
    'castellon costa azahar': 'CDT',
    'nashville metropolitan': 'BNA',
    'brasilia': 'BSB',
    'st kitts': 'SKB',
    'brest belarus': 'BQT',
    'charleston': 'CHS',
    'enfidha - hammamet intl': 'NBE',
    'antigua': 'ANU',
    'argyle international': 'SVD',
    'charleroi': 'CRL',
    'wuhan tianhe international': 'WUH',
    'jerba': 'DJE',
    'atlanta': 'ATL',
    'lyonbron': 'LYN',
    'guyancourt': 'GUY',  # Small airfield, no IATA
    'cochin': 'COK',
    'dresden': 'DRS',
    'leipzig': 'LEJ',
    'lasham': 'QLA',
    'azores ponta delgada': 'PDL',
    'jeddah': 'JED',
    'quimper': 'UIP',
    'delhi': 'DEL',
    'kosice': 'KSC',
    'dubrovnik': 'DBV',
    'liberia': 'LIR',
    'bilbao': 'BIO',
    'santiago de chile': 'SCL',
    'bastia': 'BIA',
    'unknown': 'UNK',  # Data error
    'chambery': 'CMF',
    'aarhus tirstrup': 'AAR',
    'shannon': 'SNN',
    'sondre stromfjord': 'SFJ',
    'ahmedabad': 'AMD',
    'genoa': 'GOA',
    'badajoz': 'BJZ',
    'raleigh': 'RDU',
    'columbia': 'CAE',
    'hong kong chek lap kok': 'HKG',
    'abu dhabi international': 'AUH',
    'chengdu tianfu': 'TFU',
    'suceava': 'SCV',
    'denver international': 'DEN',
    'vagar': 'FAE',
    'bodo': 'BOO',
    'donegal': 'CFN',
    'goa': 'GOI',
    'palermo': 'PMO',
    'georgetown guyana': 'GEO',
    'kittila': 'KTT',
    'blackpool': 'BLK',
    'iasi': 'IAS',
    'pescara': 'PSR',
    'moscow domodedovo': 'DME',
    'st lucia hewanorra': 'UVF',
    'santa cruz de la palma': 'SPC',
    'friedrichshafen': 'FDH',
    'payerne': 'LSMP',  # Military/civil airfield, no IATA
    'lubeck': 'LBC',
    'heraklion': 'HER',
    'hangzhou': 'HGH',
    'qingdao': 'TAO',
    'toulouse blagnac': 'TLS',
    'baku heyder aliyev intl': 'GYD',
    'jorge chavez international': 'LIM',
    'addis ababa': 'ADD',
    'beijing daxing international airport': 'PKX',
    'leon': 'LEN',
    'karup': 'KRP',
    'st etienne': 'EBU',
    'paderborn': 'PAD',
    'pula': 'PUY',
    'zadar': 'ZAD',
    'fort lauderdale': 'FLL',
    'turku': 'TKU',
    'biggin hill': 'BQH',
    'kolkata': 'CCU',
    'white plains': 'HPN',
    'baghdad geca': 'BGW',
    'bratislava': 'BTS',
    'tunis': 'TUN',
    'kristiansund kuernberget': 'KSU',
    'city of derry eglinton': 'LDY',
    'st maarten': 'SXM',
    'new york jf kennedy': 'JFK',
    'szczecin golenow': 'SZZ',
    'portsmouth usa': 'PSM',
    'figari': 'FSC',
    'colombo': 'CMB',
    'murcia international': 'RMU',
    'conakry': 'CKY',
    'sharm el sheikh ophira': 'SSH',
    'oradea': 'OMR',
    'mauritius': 'MRU',
    'tuzla': 'TZL',
    'sao tome': 'TMS',
    'zhengzhou xinzheng': 'CGO',
    'evenes': 'EVE',
    'lands end st just': 'LEQ',
    'malmo': 'MMX',
    'sharjah': 'SHJ',
    'hahn': 'HHN',
    'TATRY-POPRAD': 'TAT',
    'brussels': 'BRU',
    'xian xianyang': 'XIY',
    'abu dhabi - bateen': 'AZI',
    'khartoum': 'KRT',
    'chicago ohare': 'ORD',
    'milan linate': 'LIN',
    'charlotte': 'CLT',
    'new york newark': 'EWR',
    'comiso': 'CIY',
    'trapani': 'TPS',
    'tianjin': 'TSN',
    'whalsay': 'WHS',  # Small airstrip, no IATA
    'banjul': 'BJL',
    'austin bergstrom': 'AUS',
    'benazir bhutto international airport': 'ISB',
    'rio de janeiro galeao': 'GIG',
    'castellón costa azahar': 'CDT',
    'buffalo': 'BUF',
    'istanbul': 'IST',  # Default to new Istanbul Airport
    'grenoble': 'GNB',
    'edmonton': 'YEG',
    'kavala': 'KVA',
    'cologne bonn': 'CGN',
    'kristiansand kjevik': 'KRS',
    'pittsburgh': 'PIT',
    'azores lajes terceira island': 'TER',
    'nursultan nazerbayev international': 'NQZ',
    'tirana': 'TIA',
    'montreal dorval': 'YUL',
    'istanbul ataturk': 'IST',  # Closed, use IST for new airport
    'tokyo haneda': 'HND',
    'newburghusa': 'SWF',
    'melbourne flats': 'MLB',  # Likely Melbourne, Florida
    'tallin': 'TLL',
    'isles of scilly stmarys': 'ISC',
    'lemnos': 'LXS',
    'stockholm bromma': 'BMA',
    'podgorica': 'TGD',
    'cairo': 'CAI',
    'erfurt': 'ERF',
    'hawarden': 'CEG',
    'rodez': 'RDZ',
    'auxerre': 'AUF',
    'montpellier': 'MPL',
    'hanover': 'HAJ',
    'dakar': 'DSS',  # Likely Blaise Diagne
    'ankara esenboga': 'ESB',
    'nea anchialos': 'VOL',
    'aalborg': 'AAL',
    'burgos': 'RGS',
    'trabzon': 'TZX',
    'bucharest baneasa': 'BBU',
    'curacao': 'CUR',
    'alesund': 'AES',
    'san jose': 'SJO',  # Likely Costa Rica
    'cukurova': 'COV',  # New airport, no IATA yet
    'trieste ronchi dei legionari': 'TRS',
    'bangor': 'BGR',
    'debrecen': 'DEB',
    'west palm beach': 'PBI',
    'valladolid': 'VLL',
    'berne': 'BRN',
    'olbia': 'OLB',
    'kingston': 'KIN',
    'hurghada': 'HRG',
    'hanoi': 'HAN',
    'egilsstadir': 'EGS',
    'san francisco': 'SFO',
    'jonkoping': 'JKG',
    'salerno costa amalfi': 'QSR',
    'bari palese': 'BRI',
    'chateauroux deols': 'CHR',
    'amman king hussein': 'ADJ',
    'montego bay': 'MBJ',
    'kayseri erkilet': 'ASR',
    'ras al khaimah': 'RKT',
    'bremen': 'BRE',
    'graz': 'GRZ',
    'campbeltown': 'CAL',
    'male international': 'MLE',
    'ancona': 'AOI',
    'kumasi': 'KMS',
    'dothan alabama': 'DHN',
    'hassi messaoud': 'HME',
    'lee-on-solent': 'LEE',  # Small airfield, no IATA
    'paris le bourget': 'LBG',
    'plovdiv': 'PDV',
    'stornoway': 'SYY',
    'tangiers ibn batuta': 'TNG',
    'san diego': 'SAN',
    'gloucestershire': 'GLO',
    'phoenix': 'PHX',
    'shenzhen huangtian': 'SZX',
    'sphinx international': 'SPX',
    'molde': 'MOL',
    'brize norton': 'BZZ',
    'funchal': 'FNC',
    'madrid': 'MAD',
    'quebec': 'YQB',
    'moenchengladbach': 'MGL',
    'dalian zhoushuizi': 'DLC',
    'dusseldorf': 'DUS',
    'alghero fertilia': 'AHO',
    'vannes-meucon': 'VNE',
    'stockholm arlanda': 'ARN',
    'mumbai': 'BOM',
    'redhill': 'KRH',
    'tashkent': 'TAS',
    'ljubljana': 'LJU',
    'timisoara': 'TSR',
    'los angeles international': 'LAX',
    'diego gargia': 'DGA',  # Typo for Diego Garcia, no IATA
    'bangkok suvarnabhumi': 'BKK',
    'ohrid': 'OHD',
    'tirgu mures': 'TGM',
    'salzburg': 'SZG',
    'liege': 'LGG',
    'rimini': 'RMI',
    'linz': 'LNZ',
    'bolzano': 'BZO',
    'sanya phoenix international': 'SYX',
    'rijeka': 'RJK',
    'lille': 'LIL',
    'ostrava': 'OSR',
    'karlovy vary': 'KLV',
    'tehran imam khomeini': 'IKA',
    'granada': 'GRX',
    'bangkok don muang': 'DMK',
    'niederrhein': 'NRN',
    'pisa': 'PSA',
    'katowice': 'KTW',
    'neom bay': 'NUM',
    'keflavik': 'KEF',
    'halifax int': 'YHZ',
    'poitiers': 'PIS',
    'cluj napoca': 'CLJ',
    'calgary': 'YYC',
    'islay': 'ILY',
    'kiev borispol': 'KBP',
    'houston': 'IAH',
    'poznan': 'POZ',
    'dakar blaise diagne': 'DSS',
    'geneva': 'GVA',
    'cagliari elmas': 'CAG',
    'new orleans': 'MSY',
    'avignon': 'AVN',
    'gaziantep': 'GZT',
    'st johns': 'YYT',
    'port of spain': 'POS',
    'boston': 'BOS',
    'rabat': 'RBA',
    'berlin brandenburg': 'BER',
    'providence': 'PVD',
    'vasteras': 'VST',
    'shanghai pu dong': 'PVG',
    'singapore': 'SIN',
    'zaragoza': 'ZAZ',
    'las vegas': 'LAS',
    'ndjamena': 'NDJ',
    'kansas city': 'MCI',
    'banja luka': 'BNX',
    'san jose cost rica': 'SJO',
    'stuttgart': 'STR',
    'provenciales': 'PLS',
    'asturias': 'OVD',
    'las palmas': 'LPA',
    'hamburg': 'HAM',
    'oporto portugal': 'OPO',
    'moscow sheremetyevo': 'SVO',
    'rome fiumicino': 'FCO',
    'bergerac': 'EGC',
    'calvi': 'CLY',
    'dunkeswell': 'DNK',  # Small airfield, no IATA
    'izmir adnan menderes': 'ADB',
    'cancun': 'CUN',
    'vancouver': 'YVR',
    'nagoya': 'NGO',
    'pristina': 'PRN',
    'lvov': 'LWO',
    'lilongwe': 'LLW',
    'barra': 'BRR',
    'parma': 'PMF',
    'murcia san javier': 'MJV',
    'chennai': 'MAA',
    'mikonos': 'JMK',
    'saint helena': 'HLE',
    'rotterdam': 'RTM',
    'northolt': 'NHT',
    'lyon': 'LYS',
    'preveza': 'PVK',
    'roskilde': 'RKE',
    'perugia': 'PEG',
    'ottawa international': 'YOW',
    'fez': 'FEZ',
    'constanta': 'CND',
    'algiers': 'ALG',
    'brno turany': 'BRQ',
    'dubai': 'DXB',
    'sydney': 'SYD',
    'nimes': 'FNI',
    'dortmund': 'DTM',
    'treviso': 'TSF',
    'halmstad': 'HAD',
    'bresciamontichiari': 'VBS',
    'chongqing jiangbei international': 'CKG',
    'altenrhein': 'ACH',
    'orebro': 'ORB',
    'limoges': 'LIG',
    'chalons vatry': 'XCR',
    'taipei': 'TPE',
    'agadir al massira': 'AGA',
    'klagenfurt': 'KLU',
    'mexico city': 'MEX',
    'santander': 'SDR',
    'karlsruhebaden baden': 'FKB',
    'santiago de compostela spain': 'SCQ',
    'lagos': 'LOS',
    'a coruna': 'LCG',
    'marrakesh': 'RAK',
    'seattle tacoma': 'SEA',
    'carcassonne': 'CCF',
    'luxor': 'LXR',
    'kuwait': 'KWI',
    'BRASOV-GHIMBAV': 'GHV',
    'bucharest otopeni': 'OTP',
    'beja': 'BYJ',
    'luxembourg': 'LUX',
    'haugesund': 'HAU',
    'LUBLIN (PORT LOTNICZY)': 'LUZ',
    'moscow (domodedovo)': 'DME',
    'SAO PAULO (GUARULHOS)': 'GRU',
    'ST LUCIA (HEWANORRA)': 'UVF',
    'BODRUM (MILAS)': 'BJV',
    'lee-on-solent': 'LEE',  # Small airfield, no IATA
    'PARIS (LE BOURGET)': 'LBG',
    'TANGIERS (IBN BATUTA)': 'TNG',
    'TRONDHEIM (VAERNES)': 'TRD',
    'TOULOUSE (BLAGNAC)': 'TLS',
    'baku (heyder aliyev int\'l)': 'GYD',
    'PERTH (AUSTRALIA)': 'PER',
    'SHENZHEN (HUANGTIAN)': 'SZX',
    'OXFORD (KIDLINGTON)': 'OXF',
    'OSLO (GARDERMOEN)': 'OSL',
    'SANDEFJORD(TORP)': 'TRF',
    'ALGHERO (FERTILIA)': 'AHO',
    'MARHAM(KINGS LYNN)': 'KNF',  # RAF Marham
    'TOULON / HYERES': 'TLN',
    'vannes-meucon': 'VNE',
    'STOCKHOLM (ARLANDA)': 'ARN',
    'BRIVE-LA-GAILLARDE': 'BVE',
    'baghdad (geca)': 'BGW',
    'PARIS (CHARLES DE GAULLE)': 'CDG',
    'IRELAND WEST(KNOCK)': 'NOC',
    'kristiansund (kuernberget)': 'KSU',
    'WARSAW (MODLIN MASOVIA)': 'WMI',
    'moline (quad city)': 'MLI',
    'CITY OF DERRY (EGLINTON)': 'LDY',
    'NEW YORK (JF KENNEDY)': 'JFK',
    'INGOLSTADT-MANCHING': 'IGS',
    'SZCZECIN (GOLENOW)': 'SZZ',
    'PORTSMOUTH (USA)': 'PSM',
    'SHARM EL SHEIKH (OPHIRA)': 'SSH',
    'PARIS (ORLY)': 'ORY',
    'SEOUL (INCHEON)': 'ICN',
    'bangkok (don muang)': 'DMK',
    'LANDS END (ST JUST)': 'LEQ',
    'BOA VISTA (RABIL)': 'BVC',
    'WARSAW (CHOPIN)': 'WAW',
    'TATRY-POPRAD': 'TAT',
    'CHISINAU (KISHINEV)': 'KIV',
    'ROME (CIAMPINO)': 'CIA',
    'kiev (borispol)': 'KBP',
    'ABU DHABI - BATEEN': 'AZI',
    'chicago (o\'hare)': 'ORD',
    'CAGLIARI (ELMAS)': 'CAG',
    'MILAN (LINATE)': 'LIN',
    'NEW YORK (NEWARK)': 'EWR',
    'SHANGHAI (PU DONG)': 'PVG',
    'CATANIA (FONTANAROSSA)': 'CTA',
    'TARBES-LOURDES INTERNATIONAL': 'LDE',
    'AUSTIN (BERGSTROM)': 'AUS',
    'provenciales': 'PLS',
    'RIO DE JANEIRO (GALEAO)': 'GIG',
    'LAMETIA-TERME': 'SUF',
    'kiev (zhulyany)': 'IEV',
    'GOTEBORG (LANDVETTER)': 'GOT',
    'KRISTIANSAND (KJEVIK)': 'KRS',
    'OPORTO (PORTUGAL)': 'OPO',
    'moscow (sheremetyevo)': 'SVO',
    'ROME (FIUMICINO)': 'FCO',
    'washington (dulles)': 'IAD',
    'BANGALORE (BENGALURU)': 'BLR',
    'SZYMANY (MAZURY)': 'SZY',
    'THIRA (SANTORINI)': 'JTR',
    'IZMIR (ADNAN MENDERES)': 'ADB',
    'MONTREAL (DORVAL)': 'YUL',
    'TOKYO (HANEDA)': 'HND',
    'ISLES OF SCILLY (ST.MARYS)': 'ISC',
    'stockholm (bromma)': 'BMA',
    'LONGYEARBYEN (SVALBARD)': 'LYR',
    'ENFIDHA - HAMMAMET INTL': 'NBE',
    'HYDERABAD ( RAJIV GHANDI )': 'HYD',
    'BRNO (TURANY)': 'BRQ',
    'MUNSTER-OSNABRUCK': 'FMO',
    'ANKARA (ESENBOGA)': 'ESB',
    'TENERIFE (SURREINA SOFIA)': 'TFS',
    'BUCHAREST (BANEASA)': 'BBU',
    'TRIESTE (RONCHI DEI LEGIONARI)': 'TRS',
    'PORTLAND (OREGON)': 'PDX',
    'ROSTOCK-LAAGE': 'RLG',
    'CHALONS (VATRY)': 'XCR',
    'AGADIR (AL MASSIRA)': 'AGA',
    'AARHUS (TIRSTRUP)': 'AAR',
    'MINNEAPOLIS-ST PAUL': 'MSP',
    'SANTIAGO DE COMPOSTELA (SPAIN)': 'SCQ',
    'HONG KONG (CHEK LAP KOK)': 'HKG',
    'SEATTLE (TACOMA)': 'SEA',
    'BARI (PALESE)': 'BRI',
    'MILAN (MALPENSA)': 'MXP',
    'KUALA LUMPUR (SEPANG)': 'KUL',
    'brasov-ghimbav': 'GHV',
    'GEORGETOWN (GUYANA)': 'GEO',
    "BAKU (HEYDER ALIYEV INT'L)": "GYD",
    "CHICAGO (O'HARE)": "ORD",
    "PROVIDENCIALES": "PLS",
    'BUCHAREST (OTOPENI)': 'OTP'
}

# Define country name to ISO 3166-1 alpha-3 code mapping (expanded)
country_mapping = {
    'canada': 'CAN',
    'poland': 'POL',
    'united kingdom': 'GBR',
    'denmark': 'DNK',
    'irish republic': 'IRL',
    'netherlands': 'NLD',
    'italy': 'ITA',
    'norway': 'NOR',
    'portugal(excluding madeira)': 'PRT',
    'portugal': 'PRT',
    'spain': 'ESP',
    'turkey': 'TUR',
    # Additional countries based on destinations
    'greece': 'GRC',
    'cyprus': 'CYP',
    'bulgaria': 'BGR',
    'croatia': 'HRV',
    'france': 'FRA',
    'germany': 'DEU',
    'hungary': 'HUN',
    'iceland': 'ISL',
    'malta': 'MLT',
    'romania': 'ROU',
    'sweden': 'SWE',
    'switzerland': 'CHE',
    'austria': 'AUT',
    'belgium': 'BEL',
    'czech republic': 'CZE',
    'finland': 'FIN',
    'slovakia': 'SVK',
    'slovenia': 'SVN',
    'slovak republic': 'SVK',
    'sudan': 'SDN',
    'malaysia': 'MYS',
    'iran': 'IRN',
    'mexico': 'MEX',
    'turks and caicos islands': 'TCA',
    'bangladesh': 'BGD',
    'brunei': 'BRN',
    'grenada': 'GRD',
    'russia': 'RUS',
    'ukraine': 'UKR',
    'isle of diego garcia': 'DGA',  # Non-standard, UK/US territory
    'republic of montenegro': 'MNE',
    'costa rica': 'CRI',
    'st lucia': 'LCA',
    'saint kitts and nevis': 'KNA',
    'belarus': 'BLR',
    'armenia': 'ARM',
    'bahrain': 'BHR',
    'saudi arabia': 'SAU',
    'turkmenistan': 'TKM',
    'macedonia': 'MKD',
    'greenland': 'GRL',
    'ethiopia': 'ETH',
    'senegal': 'SEN',
    'uzbekistan': 'UZB',
    'chile': 'CHL',
    'china': 'CHN',
    'gibraltar': 'GIB',
    'malawi': 'MWI',
    'albania': 'ALB',
    'azerbaijan': 'AZE',
    'mauritius': 'MUS',
    'tunisia': 'TUN',
    'guyana': 'GUY',
    'usa': 'USA',
    'ascension island': 'ASC',
    'bosnia-herzegovina': 'BIH',
    'lithuania': 'LTU',
    'antigua and barbuda': 'ATG',
    'peru': 'PER',
    'cuba': 'CUB',
    'zimbabwe': 'ZWE',
    'gambia': 'GMB',
    'philippines': 'PHL',
    'bermuda': 'BMU',
    'cape verde islands': 'CPV',
    'taiwan': 'TWN',
    'australia': 'AUS',
    'bahamas': 'BHS',
    'republic of serbia': 'SRB',
    'georgia': 'GEO',
    'japan': 'JPN',
    'india': 'IND',
    'singapore': 'SGP',
    'isle of curacao nethantilles': 'CUW',
    'sao tome islands': 'STP',
    'rwanda': 'RWA',
    'republic of korea': 'KOR',
    'dominican republic': 'DOM',
    'trinidad and tobago': 'TTO',
    'jamaica': 'JAM',
    'republic of moldova': 'MDA',
    'egypt': 'EGY',
    'iraq': 'IRQ',
    'lebanon': 'LBN',
    'maldive islands': 'MDV',
    'kenya': 'KEN',
    'sri lanka': 'LKA',
    'suriname': 'SUR',
    'cayman islands': 'CYM',
    'argentina': 'ARG',
    'latvia': 'LVA',
    'brazil': 'BRA',
    'estonia': 'EST',
    'ghana': 'GHA',
    'kazakhstan': 'KAZ',
    'vietnam': 'VNM',
    'israel': 'ISR',
    'faroe islands': 'FRO',
    'pakistan': 'PAK',
    'united arab emirates': 'ARE',
    'thailand': 'THA',
    'republic of south africa': 'ZAF',
    'hong kong': 'HKG',
    'barbados': 'BRB',
    'nigeria': 'NGA',
    'unknown': 'UNK',  # Data error
    'oman': 'OMN',
    'colombia': 'COL',
    'algeria': 'DZA',
    'saint vincent and the grenadines': 'VCT',
    'jordan': 'JOR',
    'kosovo': 'XKX',  # Non-standard, ISO candidate
    'kuwait': 'KWT',
    'guinea': 'GIN',
    'chad': 'TCD',
    'luxembourg': 'LUX',
    'morocco': 'MAR',
    'qatar': 'QAT',
    'bosnia-herzegovina': 'BIH',
    "AARHUS (TIRSTRUP)": "AAR",
    "ABU DHABI - BATEEN": "AZI",
    "AGADIR (AL MASSIRA)": "AGA",
    "ALGHERO (FERTILIA)": "AHO",
    "ANKARA (ESENBOGA)": "ESB",
    "AUSTIN (BERGSTROM)": "AUS",
    "BAGHDAD (GECA)": "BGW",
    "BAKU (HEYDER ALIYEV INT'L)": "GYD",
    "BANGALORE (BENGALURU)": "BLR",
    "BANGKOK (DON MUANG)": "DMK",
    "BARI (PALESE)": "BRI",
    "BOA VISTA (RABIL)": "BVC",
    "BODRUM (MILAS)": "BJV",
    "BRASOV-GHIMBAV": "GHV",
    "BRIVE-LA-GAILLARDE": "BVE",
    "BRNO (TURANY)": "BRQ",
    "BUCHAREST (BANEASA)": "BBU",
    "BUCHAREST (OTOPENI)": "OTP",
    "CAGLIARI (ELMAS)": "CAG",
    "CATANIA (FONTANAROSSA)": "CTA",
    "CHALONS (VATRY)": "XCR",
    "CHICAGO (O'HARE)": "ORD",
    "CHISINAU (KISHINEV)": "KIV",
    "CITY OF DERRY (EGLINTON)": "LDY",
    "CPT:": "CPT",
    "ENFIDHA - HAMMAMET INTL": "NBE",
    "GEORGETOWN (GUYANA)": "GEO",
    "GOTEBORG (LANDVETTER)": "GOT",
    "HONG KONG (CHEK LAP KOK)": "HKG",
    "HYDERABAD ( RAJIV GHANDI )": "HYD",
    "INGOLSTADT-MANCHING": "IGS",
    "IRELAND WEST(KNOCK)": "NOC",
    "ISLES OF SCILLY (ST.MARYS)": "ISC",
    "IZMIR (ADNAN MENDERES)": "ADB",
    "KIEV (BORISPOL)": "KBP",
    "KIEV (ZHULYANY)": "IEV",
    "KRISTIANSAND (KJEVIK)": "KRS",
    "KRISTIANSUND (KUERNBERGET)": "KSU",
    "KUALA LUMPUR (SEPANG)": "KUL",
    "LAMETIA-TERME": "SUF",
    "LANDS END (ST JUST)": "LEQ",
    "LEE-ON-SOLENT": "LEE",
    "LONGYEARBYEN (SVALBARD)": "LYR",
    "LSMP": "UNK",
    "LUBLIN (PORT LOTNICZY)": "LUZ",
    "MARHAM(KINGS LYNN)": "KNF",
    "MILAN (LINATE)": "LIN",
    "MILAN (MALPENSA)": "MXP",
    "MINNEAPOLIS-ST PAUL": "MSP",
    "MOLINE (QUAD CITY)": "MLI",
    "MONTREAL (DORVAL)": "YUL",
    "MOSCOW (DOMODEDOVO)": "DME",
    "MOSCOW (SHEREMETYEVO)": "SVO",
    "MUNSTER-OSNABRUCK": "FMO",
    "NEW YORK (JF KENNEDY)": "JFK",
    "NEW YORK (NEWARK)": "EWR",
    "OPORTO (PORTUGAL)": "OPO",
    "OSLO (GARDERMOEN)": "OSL",
    "OXFORD (KIDLINGTON)": "OXF",
    "PARIS (CHARLES DE GAULLE)": "CDG",
    "PARIS (LE BOURGET)": "LBG",
    "PARIS (ORLY)": "ORY",
    "PERTH (AUSTRALIA)": "PER",
    "PORTLAND (OREGON)": "PDX",
    "PORTSMOUTH (USA)": "PSM",
    "PROVIDENCIALES": "PLS",
    "RIO DE JANEIRO (GALEAO)": "GIG",
    "ROME (CIAMPINO)": "CIA",
    "ROME (FIUMICINO)": "FCO",
    "ROSTOCK-LAAGE": "RLG",
    "SANDEFJORD(TORP)": "TRF",
    "SANTIAGO DE COMPOSTELA (SPAIN)": "SCQ",
    "SAO PAULO (GUARULHOS)": "GRU",
    "SEATTLE (TACOMA)": "SEA",
    "SEOUL (INCHEON)": "ICN",
    "SHANGHAI (PU DONG)": "PVG",
    "SHARM EL SHEIKH (OPHIRA)": "SSH",
    "SHENZHEN (HUANGTIAN)": "SZX",
    "ST LUCIA (HEWANORRA)": "UVF",
    "STOCKHOLM (ARLANDA)": "ARN",
    "STOCKHOLM (BROMMA)": "BMA",
    "SZCZECIN (GOLENOW)": "SZZ",
    "SZYMANY (MAZURY)": "SZY",
    "TANGIERS (IBN BATUTA)": "TNG",
    "TARBES-LOURDES INTERNATIONAL": "LDE",
    "TATRY-POPRAD": "TAT",
    "TENERIFE (SURREINA SOFIA)": "TFS",
    "THIRA (SANTORINI)": "JTR",
    "TOKYO (HANEDA)": "HND",
    "TOULON / HYERES": "TLN",
    "TOULOUSE (BLAGNAC)": "TLS",
    "TRIESTE (RONCHI DEI LEGIONARI)": "TRS",
    "TRONDHEIM (VAERNES)": "TRD",
    "VANNES-MEUCON": "VNE",
    "WARSAW (CHOPIN)": "WAW",
    "WARSAW (MODLIN MASOVIA)": "WMI",
    "WASHINGTON (DULLES)": "IAD"
}

# Define airline name to ICAO code or abbreviation mapping (expanded)
airline_mapping = {
    'sunwing airlines inc': 'SWG',
    'wizz air': 'WZZ',
    'loganair ltd': 'LOG',
    'eastern airways': 'EZE',
    'easyjet uk ltd': 'EZY',
    'british airways plc': 'BAW',
    'aer lingus': 'EIN',
    'klm': 'KLM',
    'klm cityhopper': 'KLC',
    'sas': 'SAS',
    'alba star': 'LAV',
    'tui fly belgium': 'TFL',
    'ryanair': 'RYR',
    'tui airways ltd': 'TOM',
    'freebird airlines': 'FHY',
    # Additional airlines for broader coverage
    'jet2.com': 'EXS',
    'tui fly nordic': 'BLX',
    'virgin atlantic airways': 'VIR',
    'emirates': 'UAE',
    'qatar airways': 'QTR',
    'lufthansa': 'DLH',
    'air france': 'AFR',
    'iberia': 'IBE',
    'vueling airlines': 'VLG',
    'tap air portugal': 'TAP',
    'norwegian air shuttle': 'NAX',
    'eurowings': 'EWG',
    'blue air': 'BMS',
    'flybe': 'BEE',
    'aurigny air services': 'AUR',
    'thomas cook airlines': 'TCX',
    'monarch airlines': 'MON',
    'zimex aviation': 'IMX',
    'united airlines': 'UAL',
    'beijing capital airlines': 'CBJ',
    'shenzhen airlines': 'CSZ',
    'sparfell france': 'SPF',
    'sky express': 'SEH',
    'air canada': 'ACA',
    'atlas air inc': 'GTI',
    'lufthansa city airlines gmbh': 'CLH',
    'nyxair ou': 'NYX',
    'trade air': 'TDR',
    'nouvelair tunisie': 'LBT',
    'abelag aviation': 'AAB',
    'qantas': 'QFA',
    'wamos air': 'PLM',
    'azerbaijan airlines azal': 'AHY',
    'surinam airways': 'SLM',
    'airexplore': 'AXE',
    'jazeera airways kscp': 'JZR',
    'albawings': 'AWT',
    'arkefly': 'TFL',  # Now TUI Fly Netherlands
    'emerald airlines ireland limited': 'EAI',
    'west jet airlines': 'WJA',
    'electra airways': 'EAF',
    'hop - brit air': 'HOP',
    'getjet airlines': 'GJT',
    'westair aviation': 'WAA',
    'aerolineas argentinas': 'ARG',
    'ryanair uk ltd': 'RUK',
    'united parcel service co': 'UPS',
    'norse atlantic airways as': 'NBT',
    'london executive aviation ltd': 'LNX',
    'corendon airlines': 'CAI',
    'norwegian air international': 'IBK',
    'cargojet airways': 'CJT',
    'malaysian airlines system-mas': 'MAS',
    'air hamburg': 'AHO',
    'air nostrum': 'ANE',
    'china airlines': 'CAL',
    'air baltic': 'BTI',
    'braathens regional aviation': 'SCW',
    'evelop': 'VLV',
    'italia trasporto aereo': 'ITY',
    'tianjin airlines': 'GCR',
    'jet2com ltd': 'EXS',
    'avianca colombia': 'AVA',
    'air malta': 'AMC',
    'saudi arabian airlines': 'SVA',
    'concierge u ltd': 'CGL',  # No ICAO, use abbreviation
    'oman air': 'OMA',
    'jet time': 'JTD',
    'air x charter': 'AXX',
    'airtanker services ltd': 'TOW',
    'hisky europe': 'HYS',
    'swiss airlines': 'SWR',
    'royal air maroc': 'RAM',
    'ba cityflyer ltd': 'CFE',
    'iraqi airways': 'IAW',
    'jordan aviation': 'JAV',
    'executive jet management europe': 'EJM',  # No ICAO, use abbreviation
    'kuwait airways': 'KAC',
    'badr airlines': 'BDR',
    'latam airlines': 'LAN',
    'malmo aviation': 'SCW',
    'croatia airlines': 'CTN',
    'flexjet operations malta': 'FJO',  # No ICAO, use abbreviation
    'air caraibes atlantique': 'FWI',
    'aeromexico': 'AMX',
    'la compagnie': 'DJT',
    'all nippon airways': 'ANA',
    'china eastern airlines': 'CES',
    'national air cargo': 'NCR',
    'brussels airlines': 'BEL',
    'transavia': 'TRA',
    'air corsica': 'CCM',
    'el al': 'ELY',
    'american airlines': 'AAL',
    'jota aviation ltd': 'ENZ',
    'philippine airlines': 'PAL',
    'wizz air uk ltd': 'WUK',
    'virgin atlantic international': 'VIR',
    'euroatlantic airways': 'MMZ',
    'corendon dutch airlines': 'CND',
    'blue islands limited': 'BCI',
    'juneyao airlines': 'DKH',
    'skyalps': 'SWP',
    'air serbia': 'ASL',
    'norwegian air sweden ab': 'NSZ',
    'edelweiss air': 'EDW',
    'korean air': 'KAL',
    'avies air company': 'AIA',
    'smartwings': 'TVS',
    'pegasus airlines': 'PGT',
    'volotea': 'VOE',
    'cyprus airways': 'CYP',
    'skyup airlines': 'SQP',
    'air bretagne': 'ABN',  # No ICAO, use abbreviation
    'tnt airways': 'TAY',
    'carpatair': 'KRP',
    'ba euroflyer ltd dba british airways': 'AWC',
    'copenhagen airtaxi': 'CAT',
    '2excel aviation ltd': 'BRO',
    'klasjet uab': 'KLJ',
    'asiana airlines': 'AAR',
    'air algerie': 'DAH',
    'bh air': 'BGH',
    'glock aviation gmbh': 'GCK',  # No ICAO, use abbreviation
    'sirio': 'SIO',  # No ICAO, use abbreviation
    'cathay pacific airways': 'CPA',
    'braathens international airways ab': 'BIX',
    'aeronexus': 'ARN',  # No ICAO, use abbreviation
    'aeroflot': 'AFL',
    'gulf air': 'GFA',
    'air moldova international': 'MLD',
    'iberia express': 'IBS',
    'air mauritius ltd': 'MAU',
    'fly play hf': 'FPY',
    'ita italia trasporto aereo': 'ITY',
    'enter air': 'ENT',
    'flightline sl': 'FTL',  # No ICAO, use abbreviation
    'wizz air malta': 'WMT',
    'hisky': 'HYS',
    'premium jet ag': 'PJA',  # No ICAO, use abbreviation
    'finnair': 'FIN',
    'biman bangladesh airlines': 'BBC',
    'novair': 'NVR',
    'virgin atlantic airways ltd': 'VIR',
    'german airways gmbh & co kg': 'GAW',
    'valljet': 'VLJ',  # No ICAO, use abbreviation
    'saxonair charter ltd': 'SAX',  # No ICAO, use abbreviation
    'europe airpost': 'FPO',
    'faroe jet': 'FJE',  # No ICAO, use abbreviation
    'hi fly': 'HFY',
    'flybe ltd': 'BEE',
    'emerald airlines uk ltd': 'EAG',
    'isles of scilly skybus': 'IOS',  # No ICAO, use abbreviation
    'air arabia maroc': 'MAC',
    'danish air transport': 'DTR',
    'unknown': 'UNK',  # Data error
    'srilankan airlines': 'ALK',
    'air albania shpk': 'ABN',
    'titan airways ltd': 'AWC',
    'israir ltd': 'ISR',
    'star east airlines': 'SRR',  # No ICAO, use abbreviation
    'air europa': 'AEA',
    'avcon jet ag': 'VCJ',  # No ICAO, use abbreviation
    'tam linhas aereas': 'TAM',
    'european air transport leipzig eat': 'BCS',
    'air astana': 'KZR',
    'hainan airlines': 'CHH',
    'flying group': 'FLG',  # No ICAO, use abbreviation
    'air alliance express': 'AEE',  # No ICAO, use abbreviation
    'med-view airline': 'MEV',
    'icelandair': 'ICE',
    'american trans air': 'AMT',
    'asl bv': 'ABR',
    'plus ultra lineas aereas': 'PUE',
    'bulgaria air': 'LZB',
    'hi fly malta': 'HFM',
    'eurowings luftverkehrs': 'EWG',
    'pen-avia ltd': 'PNA',  # No ICAO, use abbreviation
    'smartlynx airlines latvia': 'ART',
    'sparfell luftfahrt gmbh': 'SPF',
    'royal jordanian': 'RJA',
    'lot-polish airlines': 'LOT',
    'execujet scandinavia': 'EJC',  # No ICAO, use abbreviation
    'avanti air bedarfsflugges': 'ATV',
    'qatar executive': 'QQE',  # No ICAO, use abbreviation
    'vistajet ltd malta': 'VJT',
    'deutsche rettungsflugwacht': 'AMB',
    'condor': 'CFG',
    'western air charter inc dba jet edge': 'EDG',  # No ICAO, use abbreviation
    'thai airways international': 'THA',
    'cargolux airlines internatl': 'CLX',
    'tata sia airlines ltd vistara': 'VTI',
    'global reach aviation': 'GMR',  # No ICAO, use abbreviation
    'euroair': 'EUA',  # No ICAO, use abbreviation
    'swiss air ambulance': 'SAZ',  # No ICAO, use abbreviation
    'shaheen air': 'SAI',
    'ascend airways ltd': 'SYD',
    'china cargo airlines': 'CKK',
    'corendon airlines europe': 'CXI',
    'bamboo airways': 'BAV',
    'omni air international': 'OAE',
    'air china': 'CCA',
    'norse atlantic uk ltd': 'UBT',
    'thalair': 'TLR',  # No ICAO, use abbreviation
    'arkia': 'AIZ',
    'bristow helicopters ltd': 'BHL',  # No ICAO, use abbreviation
    'neos spa': 'NOS',
    'air alsie': 'MMD',
    'dot - danu oro transportas': 'DNU',  # No ICAO, use abbreviation
    'uzbekistan airlines': 'UZB',
    'helvetic airways': 'OAW',
    'harmony jets': 'HMJ',  # No ICAO, use abbreviation
    'crossair europe': 'ECC',
    'atlantic airways': 'FLI',
    'quick air service': 'QAS',  # No ICAO, use abbreviation
    'corsair': 'CRL',
    'wideroe flyveselskap as': 'WIF',
    'sundair gmbh': 'SDR',
    'air cairo': 'MSC',
    'aeroitalia srl': 'AZI',
    'marathon airways': 'MTO',  # No ICAO, use abbreviation
    'blue air transport aerian': 'BMS',
    'austrian airlines': 'AUA',
    'air blue': 'ABQ',
    'sata': 'SAT',
    'iran air': 'IRA',
    'gullivair': 'GUV',
    'aegean airlines': 'AEE',
    'ukraine international airlines': 'AUI',
    'wind rose aviation': 'WRC',
    'dan air aoc srl': 'JOC',
    'alk airlinesair lubo': 'VBB',
    'malta medair': 'MTM',  # No ICAO, use abbreviation
    'stobart air': 'STK',
    'jetnetherlands': 'JNL',  # No ICAO, use abbreviation
    'vistajet gmbh': 'VJH',  # No ICAO, use abbreviation
    'middle east airlines air liban s a l': 'MEA',
    'tarom': 'ROT',
    'jet aviation business jets': 'JBJ',  # No ICAO, use abbreviation
    'air asia': 'AXM',
    'federal express corp': 'FDX',
    'dc aviation': 'DCS',  # No ICAO, use abbreviation
    'jetblue airways corporation': 'JBU',
    'lan chile': 'LAN',
    'abs jet': 'ABJ',  # No ICAO, use abbreviation
    'world2fly portugal': 'WFY',  # No ICAO, use abbreviation
    'scandinavian airlines ireland ltd': 'SZS',
    'thomas cook scandanavia': 'VKG',
    'jetairfly': 'TUI',
    'tunisair': 'TAR',
    'luxair': 'LGL',
    'tuifly germany': 'TUI',
    'air peace': 'APK',
    'easyjet europe': 'EJU',
    'contactair flugdienst': 'KIS',
    'air india': 'AIC',
    'travel service kft': 'TVS',
    'airbus inter transport': 'BGA',
    'tui fly netherlands': 'TFL',
    'tyrol air ambulance': 'TAA',  # No ICAO, use abbreviation
    'west air sweden ab': 'SWN',
    'fleet air international ltd': 'FRF',
    'united arabian company': 'UAC',  # No ICAO, use abbreviation
    'delta airlines': 'DAL',
    'chalair': 'CLG',
    'gama aviation uk ltd': 'GMA',
    'easyjet switzerland': 'EZS',
    'airlec aviation': 'ALC',  # No ICAO, use abbreviation
    'acass ireland': 'ASI',  # No ICAO, use abbreviation
    'ryanair sun': 'RYS',
    'aer lingus uk ltd': 'EUK',
    'leav aviation gmbh': 'LVA',  # No ICAO, use abbreviation
    'west atlantic uk ltd': 'NPT',
    'rwandair express': 'RWD',
    'swiftair': 'SWT',
    'orbest': 'OBS',
    'asl airlines belgium': 'TAY',
    'turkmenistan airlines': 'TUA',
    'pan european air service': 'PEA',  # No ICAO, use abbreviation
    'maleth aero': 'DBT',
    'royal brunei airlines': 'RBA',
    'transavia france': 'TVF',
    'luxembourg air ambulance sa': 'LXA',  # No ICAO, use abbreviation
    'unijet sa': 'UJT',  # No ICAO, use abbreviation
    'eva air': 'EVA',
    'maersk air cargo as': 'DJM',
    'singapore airlines': 'SIA',
    'air one': 'ADH',
    'thy turkish airlines': 'THY',
    'frost air aps': 'FRO',  # No ICAO, use abbreviation
    'catreus aoc ltd': 'AOC',  # No ICAO, use abbreviation
    'japan airlines': 'JAL',
    'sun air of scandinavia': 'SUS',
    'tacv - cabo verde airlines': 'TCV',
    'liat-the caribbean airline': 'LIA',
    'eastern airlines llc': 'EAL',
    'egypt air': 'MSR',
    'fai flight- ambulance service': 'FAI',  # No ICAO, use abbreviation
    'intersky luftfahrt gmbh': 'ISK',
    'air transat': 'TSC',
    'air horizont': 'HAT',
    'comlux aviation malta': 'MLX',
    'myway airlines': 'MYW',
    'gestair executive jet': 'GES',
    'jetcom': 'JTC',  # No ICAO, use abbreviation
    'sunexpress': 'SXS',
    'smartlynx estonia': 'MYX',
    'scoot tigerair pteltd': 'TGW',
    'etf airways': 'EAF',
    'fly4 airlines green ltd': 'FYA',  # No ICAO, use abbreviation
    'jettime as': 'JTD',
    'vietnam airlines': 'HVN',
    'voluxis ltd': 'VLX',
    'hex air': 'HER',
    'privilege style': 'PVG',
    'netjets transportes aereos': 'NJE',
    'aeronova': 'OVA',
    'aero caribbean': 'CRN',
    'etihad airways': 'ETD',
    'chartright air inc': 'HRT',  # No ICAO, use abbreviation
    'ajet hava tasimaciligi anonim sirketi': 'THY',  # Operated by Turkish Airlines
    'freebird airlines europe limited': 'FHY',
    'dhl air ltd': 'DHK',
    'flyr': 'FOX',
    'efs european flight service ab': 'EFS',  # No ICAO, use abbreviation
    'timeair sro': 'TIE',  # No ICAO, use abbreviation
    'china southern': 'CSN',
    'planet nine private air': 'PNA',  # No ICAO, use abbreviation
    'aero4m': 'AEH',
    'air dolomiti': 'DLA',
    'mhs aviation gmbh gruenwald': 'MHV',
    'kenya airways': 'KQA',
    'sun express deutschland': 'SXD',
    'air portugal': 'TAP',
    'csa czech airlines': 'CSA',
    'ethiopian airlines': 'ETH',
    'acm air charter luftfaht': 'BVR',
    'fly one': 'FIA',
    'luxwing': 'LWG',
    'german airways gmbh & co kg': 'GAW',
    'malaysian airlines system-mas': 'MAS',
    'pen-avia ltd': 'PNA',  # No ICAO, abbreviation used
    'smartlynx airlines (latvia)': 'ART',
    'lot-polish airlines': 'LOT',
    'comlux aviation (malta)': 'MLX',
    '(ita) italia trasporto aereo': 'ITY',
    'smartlynx (estonia)': 'MYX',
    'tuifly (germany)': 'TUI',
    'italia trasporto aereo': 'ITY',
    'dan air (aoc) s.r.l': 'JOC',
    'aer lingus (uk) ltd': 'EUK',
    'tacv - cabo verde airlines': 'TCV',
    'emerald airlines (ireland) limited': 'EAI',
    'european air transport leipzig (eat)': 'BCS',
    'liat-the caribbean airline': 'LIA',
    'dot - danu oro transportas': 'DNU',  # No ICAO, abbreviation used
    'middle east airlines (air liban s a l)': 'MEA',
    'hop - brit air': 'HOP',
    'fai flight- ambulance service': 'FAI',  # No ICAO, abbreviation used
    'med-view airline': 'MEV',
    'gama aviation (uk) ltd': 'GMA',
    'tata sia airlines ltd (vistara)': 'VTI',
    "(ITA) ITALIA TRASPOTO AEREO": "ITY",
    "AER LINGUS (UK) LTD": "EUK",
    "AZERBAIJAN AIRLINES (AZAL)": "AHY",
    "COMLUX AVIATION (MALTA)": "MLX",
    "DAN AIR (AOC) S.R.L": "JOC",
    "DOT - DANU ORO TRANSPORTAS": "DNU",
    "EMERALD AIRLINES (IRELAND) LIMITED": "EAI",
    "EUROPEAN AIR TRANSPORT LEIPZIG (EAT)": "BCS",
    "FAI FLIGHT- AMBULANCE SERVICE": "FAI",
    "GAMA AVIATION (UK) LTD": "GMA",
    "GERMAN AIRWAYS GMBH & CO KG": "GAW",
    "HOP - BRIT AIR": "HOP",
    "ITALIA TRASPOTO AEREO": "ITY",
    "LIAT-THE CARIBBEAN AIRLINE": "LIA",
    "LOT-POLISH AIRLINES": "LOT",
    "MALAYSIAN AIRLINES SYSTEM-MAS": "MAS",
    "MED-VIEW AIRLINE": "MEV",
    "MIDDLE EAST AIRLINES (AIR LIBAN S A L)": "MEA",
    "PEN-AVIA LTD": "PNA",
    "SMARTLYNX (ESTONIA)": "MYX",
    "SMARTLYNX AIRLINES (LATVIA)": "ART",
    "TACV - CABO VERDE AIRLINES": "TCV",
    "TATA SIA AIRLINES LTD (VISTARA)": "VTI",
    "TUIFLY (GERMANY)": "TUI"
}

# Function to normalize names for mapping
def normalize_name(name):
    if pd.isna(name) or name == '':
        return ''
    name = str(name).lower().strip()
    # Remove quotes, parentheses content, and extra spaces
    name = re.sub(r'[\'"]', '', name)
    name = re.sub(r'\([^)]*\)', '', name)  # Remove content in parentheses
    name = re.sub(r'[^\w\s]', '', name)    # Remove special characters
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# Function to get code from mapping, fallback to uppercase original
def get_code(name, mapping, unmapped_set):
    original_name = name
    normalized = normalize_name(name)
    if normalized == '':
        return original_name
    code = mapping.get(normalized, original_name.upper())
    if code == original_name.upper() and normalized not in mapping:
        unmapped_set.add(original_name)
    return code

# Sets to track unmapped values
unmapped_airports = set()
unmapped_countries = set()
unmapped_airlines = set()

# Replace names with codes
df['REPORTING_AIRPORT'] = df['REPORTING_AIRPORT'].apply(lambda x: get_code(x, airport_mapping, unmapped_airports))
df['ORIGIN_DESTINATION_COUNTRY'] = df['ORIGIN_DESTINATION_COUNTRY'].apply(lambda x: get_code(x, country_mapping, unmapped_countries))
df['ORIGIN_DESTINATION'] = df['ORIGIN_DESTINATION'].apply(lambda x: get_code(x, airport_mapping, unmapped_airports))
df['AIRLINE_NAME'] = df['AIRLINE_NAME'].apply(lambda x: get_code(x, airline_mapping, unmapped_airlines))

# Output the transformed data to a new CSV
output_path = "/content/Punchuality_Stats_Fact.csv"
df.to_csv(output_path, index=False)

# Create a DataFrame for unmapped values
unmapped_data = []
for airport in unmapped_airports:
    unmapped_data.append({'Column': 'REPORTING_AIRPORT/ORIGIN_DESTINATION', 'Unmapped_Value': airport})
for country in unmapped_countries:
    unmapped_data.append({'Column': 'ORIGIN_DESTINATION_COUNTRY', 'Unmapped_Value': country})
for airline in unmapped_airlines:
    unmapped_data.append({'Column': 'AIRLINE_NAME', 'Unmapped_Value': airline})

unmapped_df = pd.DataFrame(unmapped_data)
unmapped_output_path = "/content/unmapped_values.csv"
if not unmapped_df.empty:
    unmapped_df.to_csv(unmapped_output_path, index=False)
else:
    pd.DataFrame({'Column': [], 'Unmapped_Value': []}).to_csv(unmapped_output_path, index=False)

# Download the output files
files.download(output_path)
files.download(unmapped_output_path)

# Print diagnostics
print(f"Fact table generated at: {output_path}")
print(f"Unmapped values saved at: {unmapped_output_path}")
print("Both files are being downloaded to your local machine.")
print("\nMapping Statistics:")
print(f"Unique REPORTING_AIRPORT/ORIGIN_DESTINATION values: {len(unique_airports)}")
print(f"  Mapped: {len(unique_airports) - len(unmapped_airports)}")
print(f"  Unmapped: {len(unmapped_airports)}")
print(f"Unique ORIGIN_DESTINATION_COUNTRY values: {len(unique_countries)}")
print(f"  Mapped: {len(unique_countries) - len(unmapped_countries)}")
print(f"  Unmapped: {len(unmapped_countries)}")
print(f"Unique AIRLINE_NAME values: {len(unique_airlines)}")
print(f"  Mapped: {len(unique_airlines) - len(unmapped_airlines)}")
print(f"  Unmapped: {len(unmapped_airlines)}")
if unmapped_airports:
    print("\nUnmapped airports (retained as uppercase original names):", sorted(unmapped_airports))
if unmapped_countries:
    print("Unmapped countries (retained as uppercase original names):", sorted(unmapped_countries))
if unmapped_airlines:
    print("Unmapped airlines (retained as uppercase original names):", sorted(unmapped_airlines))
print("\nCheck 'unmapped_values.csv' for the full list of unmapped values.")



Loaded columns: ['REPORTING_PERIOD', 'REPORTING_AIRPORT', 'ORIGIN_DESTINATION_COUNTRY', 'ORIGIN_DESTINATION', 'AIRLINE_NAME', 'FLIGHT_TYPE', 'FLIGHTS_MATCHED', 'FLIGHTS_CANCELLED', 'AVERAGE_DELAY', 'PY_MONTH_FLIGHTS_MATCHED', 'PY_MONTH_AVERAGE_DELAY', 'CATEGORY', 'value']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fact table generated at: /content/Punchuality_Stats_Fact.csv
Unmapped values saved at: /content/unmapped_values.csv
Both files are being downloaded to your local machine.

Mapping Statistics:
Unique REPORTING_AIRPORT/ORIGIN_DESTINATION values: 592
  Mapped: 508
  Unmapped: 84
Unique ORIGIN_DESTINATION_COUNTRY values: 122
  Mapped: 121
  Unmapped: 1
Unique AIRLINE_NAME values: 275
  Mapped: 0
  Unmapped: 275

Unmapped airports (retained as uppercase original names): ['AARHUS (TIRSTRUP)', 'ABU DHABI - BATEEN', 'AGADIR (AL MASSIRA)', 'ALGHERO (FERTILIA)', 'ANKARA (ESENBOGA)', 'AUSTIN (BERGSTROM)', "BAKU (HEYDER ALIYEV INT'L)", 'BANGALORE (BENGALURU)', 'BARI (PALESE)', 'BOA VISTA (RABIL)', 'BODRUM (MILAS)', 'BRASOV-GHIMBAV', 'BRIVE-LA-GAILLARDE', 'BRNO (TURANY)', 'BUCHAREST (BANEASA)', 'BUCHAREST (OTOPENI)', 'CAGLIARI (ELMAS)', 'CATANIA (FONTANAROSSA)', 'CHALONS (VATRY)', "CHICAGO (O'HARE)", 'CHISINAU (KISHINEV)', 'CITY OF DERRY (EGLINTON)', 'ENFIDHA - HAMMAMET INTL', 'GEORGETOWN (GUYANA)'